In [1]:
import numpy as np
import numpy.random as nrand
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import itertools
from itertools import combinations
import copy
from matplotlib.ticker import FormatStrFormatter
from sklearn.linear_model import Ridge
from scipy.linalg import hadamard

In /home/siliang/usr/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/siliang/usr/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/siliang/usr/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
def load_sequence(sequences):
    BASES = np.asarray(['A','T','C','G'])
    shape = sequences.shape
    data = sequences[..., None] == BASES
    return data.reshape(shape[0], shape[1] * BASES.size)

In [3]:
def get_fitness(read_count_in, read_count_out, in_idx, out_idx):
    OD_out = np.array([
        [1.39,1.21,1.30],
        [1.17,1.23,1.14]
    ])
    OD_innitial = 0.015
    wt_read_count_in = df_all[f'reads_IN_{in_idx}'][0]
    wt_read_count_out = df_all[f'reads_OUT_{in_idx}{out_idx}'][0]
    total_read_count_in = df_all[f'reads_IN_{in_idx}'].sum()
    total_read_count_out = df_all[f'reads_OUT_{in_idx}{out_idx}'].sum()
    
    f_in = OD_innitial*read_count_in/total_read_count_in
    f_out = OD_out[in_idx-1,out_idx-1]*read_count_out/total_read_count_out
    
    f_in_wt = OD_innitial*wt_read_count_in/total_read_count_in
    f_out_wt = OD_out[in_idx-1,out_idx-1]*wt_read_count_out/total_read_count_out
    
    n = np.log2(f_out/f_in) # number of generation
    n_wt = np.log2(f_out_wt/f_in_wt)
    
    return np.log(n/n_wt)
    

In [4]:
df_all = pd.read_csv('trna_Domingo_data.csv')
seqs_raw = df_all.seq.to_numpy()
seqs_raw = np.array(list(map(list,seqs_raw)))
seqs = seqs_raw[:,np.array([1,2,6,27,43,46,66,69,70,71])-1]

In [5]:
x = load_sequence(seqs)
x = x[:,np.where((x != x[0]).sum(axis=0) > 0)[0]]

In [6]:
# Calculate fitness

fitness_dict = {f'Fit{i+1}{j+1}':[] for i in range(2) for j in range(3)}
for idx, row in df_all.iterrows():
    for in_idx in [1,2]:
        for out_idx in [1,2,3]:
            read_count_in = row[f'reads_IN_{in_idx}']
            read_count_out = row[f'reads_OUT_{in_idx}{out_idx}']
            fitness = get_fitness(read_count_in, read_count_out, in_idx, out_idx)
            fitness_dict[f'Fit{in_idx}{out_idx}'].append(fitness)

fitness_df = pd.DataFrame(fitness_dict)

In [7]:
# choose genotype subspace.
# 0:G/A 1:U/G; 2:G/U; 3:C/U, 4:A/C, 5:C/U; 6:C/U; 7:A/G; 8:G/U; 9:C/U; with only 1 missing genotype 
# 1,2,6,8 are tri-allelic
seqs_sub = seqs[(seqs[:,1] != 'C') & (seqs[:,2] != 'A') & (seqs[:,6] != 'A') & (seqs[:,8] != 'A')]
fitness_df_sub = fitness_df[(seqs[:,1] != 'C') & (seqs[:,2] != 'A') & (seqs[:,6] != 'A') & (seqs[:,8] != 'A')]
seqs_sub = (seqs_sub != ['G','U','G','C','A','C','C','A','G','C']).astype(int)
seq_fit_dict = {''.join(seqs_sub[i].astype('str')): fitness_df_sub.iloc[i].to_list() for i in range(len(seqs_sub))}

N=10
landscape_list_b = []
for i in range(2**N):
    seq_b = [int(x) for x in bin(i)[2:]]
    seq_b = [0]*(N-len(seq_b))+seq_b
    landscape_list_b.append(seq_b)
landscape_list_b = np.array(landscape_list_b)

fitness_df_sub_ordered = {'Seq':[],'Fit11':[],'Fit12':[],'Fit13':[],'Fit21':[],'Fit22':[],'Fit23':[]}
for seq in landscape_list_b:
    seq_str = ''.join(seq.astype('str'))
    if seq_str in seq_fit_dict:
        r1,r2,r3,r4,r5,r6 = seq_fit_dict[seq_str]
        fitness_df_sub_ordered['Seq'].append(seq_str)
        fitness_df_sub_ordered['Fit11'].append(r1)
        fitness_df_sub_ordered['Fit12'].append(r2)
        fitness_df_sub_ordered['Fit13'].append(r3)
        fitness_df_sub_ordered['Fit21'].append(r4)
        fitness_df_sub_ordered['Fit22'].append(r5)
        fitness_df_sub_ordered['Fit23'].append(r6)
    else:
        fitness_df_sub_ordered['Seq'].append(seq_str)
        fitness_df_sub_ordered['Fit11'].append(-1)
        fitness_df_sub_ordered['Fit12'].append(-1)
        fitness_df_sub_ordered['Fit13'].append(-1)
        fitness_df_sub_ordered['Fit21'].append(-1)
        fitness_df_sub_ordered['Fit22'].append(-1)
        fitness_df_sub_ordered['Fit23'].append(-1)
        
fitness_df_sub_ordered = pd.DataFrame(fitness_df_sub_ordered)

In [8]:
# interopolate missing fitness estimations using linear model
for rep in ['Fit11','Fit12','Fit13','Fit21','Fit22','Fit23']:
    _y = fitness_df_sub[rep].to_numpy()
    _x = seqs_sub
    reg = Ridge(fit_intercept=True).fit(_x, _y)
    y_predict = reg.predict(landscape_list_b)
    missing_idx_list = np.where(fitness_df_sub_ordered[rep] == -1)[0]
    fitness_df_sub_ordered.loc[missing_idx_list,rep] = y_predict[missing_idx_list]

In [13]:
# Change the parameter to select different ruggedness measure
metric = 'r_s' # 'N_max','epi','r_s','open_ratio'

In [14]:
if metric in ['N_max','gamma']:
    with open('../../index_file/trna_Domingo_neighbor_list.pkl','rb') as f:
        neighbor_list = pickle.load(f)

if metric in ['epi','gamma']:
    with open('../../index_file/trna_Domingo_epi_square_list.pkl','rb') as f:
        epi_square_list = pickle.load(f)
    
elif metric == 'open_ratio':
    with open('../../index_file/trna_Domingo_pathway_list_4steps_all.pkl','rb') as f:
        pathway_list = pickle.load(f)
    _y = df_all.fitness.to_numpy()
    y20,y80 = np.percentile(_y,[20,80])
    filtered_ascend = (_y[pathway_list[:,0]]<=y20) & (_y[pathway_list[:,-1]]>=y80)
    filtered_descend = (_y[pathway_list[:,0]]>=y80) & (_y[pathway_list[:,-1]]<=y20)
    pathway_list = pathway_list[filtered_ascend|filtered_descend,:]

In [17]:
def get_N_max(y):
    N_max = 0
    for i in range(len(y)):
        fit = y[i]
        if np.sum(fit <= y[neighbor_list[i]])==0:
            N_max += 1
    return N_max

def cal_epi(y):
    epi_fit_list = y[epi_square_list]
    n_epi = np.sum(np.sum(epi_fit_list[:,[0,0,3,3]] > epi_fit_list[:,[1,2,1,2]],axis=1)==4)
    n_epi += np.sum(np.sum(epi_fit_list[:,[0,0,3,3]] < epi_fit_list[:,[1,2,1,2]],axis=1)==4)
    return n_epi/len(epi_fit_list)

def cal_r_s(y):
    # x is a global variable with each row being a one-hot encode sequence for the corresponding genotype.
    reg = Ridge(fit_intercept=True).fit(x, y)
    y_predict = reg.predict(x)
    roughness = np.sqrt(np.mean(np.square(y - y_predict)))
    slope = np.mean(np.abs(reg.coef_))
    return roughness/slope

def cal_open_ratio(y):
    diff = y[pathway_list[:,:-1]] - y[pathway_list[:,1:]] 
    open_descend = np.sum(np.sum(diff >= 0,axis=1) == pathway_list.shape[1]-1)
    open_ascend = np.sum(np.sum(diff <= 0,axis=1) == pathway_list.shape[1]-1)
    total_open = open_descend + open_ascend
    return total_open/len(pathway_list)

def cal_E(y):
    global idx_1, phi
    W = y.astype('float32')
    E = phi.dot(W)/(2**N)
    E_square = np.square(E)
    E_sum = E_square.sum()-E_square[0]
    E_1 = E_square[idx_1].sum()
    #E_2 = E_square[idx_2].sum()
    #F_2 = E_2/E_sum
    F_sum = (E_sum-E_1)/E_sum
    return F_sum

def cal_gamma(y):
    cov = np.sum((y[epi_square_list][:,1]-y[epi_square_list][:,0])*(y[epi_square_list][:,3]-y[epi_square_list][:,2])) +\
    np.sum((y[epi_square_list][:,2]-y[epi_square_list][:,0])*(y[epi_square_list][:,3]-y[epi_square_list][:,1]))
    cov = cov/(2*epi_square_list.shape[0])
    sg_list = []
    for i in range(len(y)):
        for neighbor_idx in neighbor_list[i]:
            sg_list.append(y[neighbor_idx] - y[i])
    var = np.var(sg_list)
    return cov/var

def cal_adptwalk_steps(y):
    N_step_list = []
    for idx_0 in range(len(y)):
        idx_current = idx_0
        N_step = 0
        while True:
            fit_current = y[idx_current]
            neighbor = neighbor_list[idx_current]
            if len(neighbor) == 0 :
                if N_step > 0: N_step_list.append(N_step)
                break
            fit_next = y[neighbor].max()
            if fit_next <= fit_current:
                if N_step > 0: N_step_list.append(N_step)
                break
            idx_next = neighbor[np.argmax(y[neighbor])]
            N_step += 1
            idx_current = idx_next
    return np.mean(N_step_list)

def cal_adptwalk_probs(y):
    idx_GO = np.argmax(y)
    N_reach = 0
    N_total = 0
    for idx_0 in range(len(y)):
        idx_current = idx_0
        N_step = 0
        while True:
            fit_current = y[idx_current]
            neighbor = neighbor_list[idx_current]
            if len(neighbor) == 0:
                if N_step > 0: N_total += 1
                break
            fit_next = y[neighbor].max()
            if fit_next <= fit_current:
                if N_step > 0:
                    N_total += 1
                    if idx_current == idx_GO:
                        N_reach += 1
                break
            idx_next = neighbor[np.argmax(y[neighbor])]
            N_step += 1
            idx_current = idx_next
    return N_reach/N_total


if metric == 'N_max':
    get_ruggedness = get_N_max

elif metric == 'epi':
    get_ruggedness = cal_epi

elif metric == 'r_s':
    get_ruggedness = cal_r_s

elif metric == 'open_ratio':
    get_ruggedness = cal_open_ratio

elif metric == 'E':
    N=10
    landscape_list_b = []
    for i in range(2**N):
        seq_b = [int(x) for x in bin(i)[2:]]
        seq_b = [0]*(N-len(seq_b))+seq_b
        landscape_list_b.append(seq_b)
    landscape_list_b = np.array(landscape_list_b)
    get_ruggedness = cal_E
    phi = hadamard(2**N,dtype='float32')
    idx_1 = landscape_list_b.sum(axis=1) == 1

elif metric == 'gamma':
    get_ruggedness = cal_gamma
    
elif metric == 'adptwalk_steps':
    get_ruggedness = cal_adptwalk_steps
    
elif metric == 'adptwalk_probs':
    get_ruggedness = cal_adptwalk_probs

In [18]:
duplicates_list = ['Fit11','Fit12','Fit13','Fit21','Fit22','Fit23']
res_dict = {i:[] for i in range(1,7)}
for replication in range(1,7):
    print(replication,end='\r')
    if replication == 1:
        for duplicate in duplicates_list:
            if metric == 'E':
                y = fitness_df_sub_ordered[duplicate].to_numpy()
            else:
                y = fitness_df[duplicate].to_numpy()
            res_dict[replication].append(get_ruggedness(y))
    else:
        iter_list = combinations(duplicates_list,replication)   
        for duplicate in iter_list:
            if metric == 'E':
                y = fitness_df_sub_ordered[list(duplicate)].mean(axis=1).to_numpy()
            else:
                y = fitness_df[list(duplicate)].mean(axis=1).to_numpy()
            res_dict[replication].append(get_ruggedness(y))
        

In [20]:
# Uncomment only if you want to overwrite trna_raw_data folder.
# with open(f'./trna_Domingo_{metric}_plot.pkl','wb') as f:
#     pickle.dump(res_dict,f)